In [1]:
# auto reload modules
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
import re
from datetime import datetime
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
import pycountry
from IPython.core.display_functions import display
from IPython.display import Markdown
from wbgapi import economy
from wordcloud import STOPWORDS
import altair as alt

from utils import preprocess_concept_df, create_choropleth_map, create_choropleth_map_co_occurrence

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
concept_1 = "fossil-fuels"
concept_2 = "financial-flows"
display(Markdown(f"# GST Co-Occurrence Report: {concept_1.replace('-', ' ').title()} and {concept_2.replace('-', ' ').title()}"))

# GST Co-Occurrence Report: Fossil Fuels and Financial Flows

## Summary

In [5]:
# set the display options to allow resizing columns
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [6]:
concepts_path = Path().absolute().parent / "concepts"

In [7]:
date_of_nb = datetime.now().strftime("%d-%m-%Y")
df_concepts_1 = pd.read_excel(concepts_path / concept_1 / "output_with_metadata.xlsx")
# df_spans_1 = pd.read_csv(concepts_path / concept_1 / "spans.csv")
date_of_nb = datetime.today().strftime("%d-%m-%Y")
df_input_1 = pd.read_excel(concepts_path / concept_1 / "input.xlsx")
df_concepts_2 = pd.read_excel(concepts_path / concept_2 / "output_with_metadata.xlsx")
df_input_2 = pd.read_excel(concepts_path / concept_2 / "input.xlsx")

In [8]:
scraper_csv_path = os.getenv("SCRAPER_CSV_PATH", "scraper_csv")

In [9]:
df_unfccc = pd.read_csv(scraper_csv_path)
df_unfccc["category"] = np.where(df_unfccc["party"].notna(), "Party", "Non-Party")

In [10]:
df_unfccc

,md5sum,document_name,theme,type,author,author_type,date,language,link,pdf_link,validation_status,data_error_type,party,translation,version,status,source,topics,category
0,7159d5e03063be2e77138fd2a8499bcf,United States of America. Biennial report (BR). BR 2.,Emission reduction targets,Biennial reports (BR),United States of America,NaN,NaN,English,/documents/198949,https://unfccc.int/sites/default/files/resource/second_biennial_report_methodologies_for_quantified_policies_and_measures.pdf,not validated,NaN,United States of America,NaN,NaN,NaN,UNFCCC Information Portal,Emission reduction targets,Party
1,072bb9e1cc693272b7e0c787d943dda3,"Biennial in-session workshop on information to be provided by Parties in accordance with Article 9, paragraph 5, of the Paris Agreement. Summary report by the secretariat",Climate finance,Summary reports,UNFCCC. Conference of the Parties serving as the meeting of the Parties to the Paris Agreement (CMA),NaN,2021-09-10T12:00:00Z,English,/documents/306727,https://unfccc.int/sites/default/files/resource/cma2021_05E.pdf,not validated,NaN,NaN,NaN,NaN,NaN,UNFCCC Information Portal,Climate finance,Non-Party
2,8d5d20f1335ff37244999ccb18825b6e,Gambia. National communication (NC). NC 3.,"Policies and measures, National communications (NC), Mitigation",National communications (NC),Gambia,NaN,2020-07-27T12:00:00Z,English,/documents/231735,https://unfccc.int/sites/default/files/resource/The%20Gambia%20Third%20National%20Communication.pdf,not validated,NaN,Gambia,NaN,NaN,NaN,UNFCCC Information Portal,"Policies and measures,National communications NC,Mitigation",Party
3,e47e24113c01ba45987a1ba69894dad7,Technical analysis of the third biennial update report of Viet Nam submitted on 16 April 2021. Summary report by the team of technical experts,"Biennial update reports (BUR), National communications (NC), International consultations and analysis (ICA), Mitigation, Means of implementation",Technical analysis summary report (TASR),UNFCCC. Secretariat,NaN,2022-08-22T12:00:00Z,English only,/documents/611238,https://unfccc.int/sites/default/files/resource/tast2021_VNM.pdf,not validated,NaN,Viet Nam,NaN,NaN,NaN,UNFCCC Information Portal,"Biennial update reports BUR,National communications NC,International consultations and analysis ICA,Mitigation,Means of implementation",Party
4,747314abd32a96da1c1bcc51ec0b1088,Technical analysis of the first biennial update report of Benin submitted on 24 October 2019. Summary report by the team of technical experts,"Biennial update reports (BUR), National communications (NC), International consultations and analysis (ICA), Means of implementation, Mitigation",Technical analysis summary report (TASR),UNFCCC. Secretariat,NaN,2020-11-12T12:00:00Z,English only,/documents/266341,https://unfccc.int/sites/default/files/resource/tasr2020_BEN.pdf,not validated,NaN,Benin,NaN,NaN,NaN,UNFCCC Information Portal,"Biennial update reports BUR,National communications NC,International consultations and analysis ICA,Means of implementation,Mitigation",Party
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,dbb7a00f5db1c8034750aca5987d1382,Zambia. National communication (NC). NC 3.,"Policies and measures, Mitigation",National communications (NC),Zambia,NaN,2020-09-24T12:00:00Z,English,/documents/254196,https://unfccc.int/sites/default/files/resource/Third%20National%20Communication%20-%20Zambia.pdf,not validated,NaN,NaN,NaN,NaN,NaN,UNFCCC Information Portal,NaN,Non-Party
1073,ec515ae139508bd1a4e4f800aa1c65d9,Palau. National communication (NC). NC2.,"Policies and measures, Mitigation",Palau. National communication (NC). NC2.,Palau,NaN,2019-08-27T12:00:00Z,English,/documents/199648,https://unfccc.int/sites/default/files/resource/Final_Palau%20National%20Communication.pdf,not validated,NaN,NaN,NaN,NaN,NaN,UNFCCC Information Portal,NaN,Non-Party
1074,af19ed11954801cd7099621cf64a509c,Montenegro. Biennial update report (BUR). BUR 2.,Mitigation actions and commitments,"Bienni

In [11]:
display(Markdown(f"### UNFCCC Party members mentioning {concept_1.replace('-', ' ').title()} and {concept_2.replace('-', ' ').title()} in submitted documents"))

### UNFCCC Party members mentioning Fossil Fuels and Financial Flows in submitted documents

In [12]:
df_eco = pd.DataFrame(economy.list())
df_worldbank = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
df_concept_1_processed = preprocess_concept_df(df_concepts_1, df_worldbank, df_eco)
df_concept_2_processed = preprocess_concept_df(df_concepts_2, df_worldbank, df_eco)

In [101]:
df_concept_1_processed["concept_group"] = 1
df_concept_2_processed["concept_group"] = 2

In [102]:
# concat the two dataframes
df_concepts_combined = pd.concat([df_concept_1_processed, df_concept_2_processed])

In [103]:
df_concepts_geoplot_1 = df_concept_1_processed.groupby(['iso_a3','Concept']).agg(
    **{'Number of mentions': ('iso_a3', 'count')},
    # Concept=('Concept', lambda x: x[x.first_valid_index()] if x.first_valid_index() else None),
    geometry=('geometry', 'first'),
    pop_est=('pop_est', 'first'),
    gdp_md_est=('gdp_md_est', 'first'),
    continent=('continent', 'first'),
    country=('country', 'first'),
).reset_index()

In [104]:
df_concepts_geoplot_1

,iso_a3,Concept,Number of mentions,geometry,pop_est,gdp_md_est,continent,country
0,AFG,Coal,68,"POLYGON ((66.51861 37.36278, 67.07578 37.35614, 67.83000 37.14499, 68.13556 37.02312, 68.85945 37.34434, 69.19627 37.15114, 69.51879 37.60900, 70.11658 37.58822, 70.27057 37.73516, 70.37630 38.13840, 70.80682 38.48628, 71.34813 38.25891, 71.23940 37.95327, 71.54192 37.90577, 71.44869 37.06564, 71.84464 36.73817, 72.19304 36.94829, 72.63689 37.04756, 73.26006 37.49526, 73.94870 37.42157, 74.98000 37.41999, 75.15803 37.13303, 74.57589 37.02084, 74.06755 36.83618, 72.92002 36.72001, 71.84629 36.50994, 71.26235 36.07439, 71.49877 35.65056, 71.61308 35.15320, 71.11502 34.73313, 71.15677 34.34891, 70.88180 33.98886, 69.93054 34.02012, 70.32359 33.35853, 69.68715 33.10550, 69.26252 32.50194, 69.31776 31.90141, 68.92668 31.62019, 68.55693 31.71331, 67.79269 31.58293, 67.68339 31.30315, 66.93889 31.30491, 66.38146 30.73890, 66.34647 29.88794, 65.04686 29.47218, 64.35042 29.56003, 64.14800 29.34082, 63.55026 29.46833, 62.54986 29.31857, 60.87425 29.82924, 61.78122 30.73585, 61.69931 31.37951, 60.94194 31.54807, 60.86365 32.18292, 60.53608 32.98127, 60.96370 33.52883, 60.52843 33.67645, 60.80319 34.40410, 61.21082 35.65007, 62.23065 35.27066, 62.98466 35.40404, 63.19354 35.85717, 63.98290 36.00796, 64.54648 36.31207, 64.74611 37.11182, 65.58895 37.30522, 65.74563 37.66116, 66.21738 37.39379, 66.51861 37.36278))",38041754.0,19291.0,Asia,Afghanistan
1,AFG,Fossil fuel,2,"POLYGON ((66.51861 37.36278, 67.07578 37.35614, 67.83000 37.14499, 68.13556 37.02312, 68.85945 37.34434, 69.19627 37.15114, 69.51879 37.60900, 70.11658 37.58822, 70.27057 37.73516, 70.37630 38.13840, 70.80682 38.48628, 71.34813 38.25891, 71.23940 37.95327, 71.54192 37.90577, 71.44869 37.06564, 71.84464 36.73817, 72.19304 36.94829, 72.63689 37.04756, 73.26006 37.49526, 73.94870 37.42157, 74.98000 37.41999, 75.15803 37.13303, 74.57589 37.02084, 74.06755 36.83618, 72.92002 36.72001, 71.84629 36.50994, 71.26235 36.07439, 71.49877 35.65056, 71.61308 35.15320, 71.11502 34.73313, 71.15677 34.34891, 70.88180 33.98886, 69.93054 34.02012, 70.32359 33.35853, 69.68715 33.10550, 69.26252 32.50194, 69.31776 31.90141, 68.92668 31.62019, 68.55693 31.71331, 67.79269 31.58293, 67.68339 31.30315, 66.93889 31.30491, 66.38146 30.73890, 66.34647 29.88794, 65.04686 29.47218, 64.35042 29.56003, 64.14800 29.34082, 63.55026 29.46833, 62.54986 29.31857, 60.87425 29.82924, 61.78122 30.73585, 61.69931 31.37951, 60.94194 31.54807, 60.86365 32.18292, 60.53608 32.98127, 60.96370 33.52883, 60.52843 33.67645, 60.80319 34.40410, 61.21082 35.65007, 62.23065 35.27066, 62.98466 35.40404, 63.19354 35.85717, 63.98290 36.00796, 64.54648 36.31207, 64.74611 37.11182, 65.58895 37.30522, 65.74563 37.66116, 66.21738 37.39379, 66.51861 37.36278))",38041754.0,19291.0,Asia,Afghanistan
2,AFG,Gas,125,"POLYGON ((66.51861 37.36278, 67.07578 37.35614, 67.83000 37.14499, 68.13556 37.02312, 68.85945 37.34434, 69.19627 37.15114, 69.51879 37.60900, 70.11658 37.58822, 70.27057 37.73516, 70.37630 38.13840, 70.80682 38.48628, 71.34813 38.25891, 71.23940 37.95327, 71.54192 37.90577, 71.44869 37.06564, 71.84464 36.73817, 72.19304 36.94829, 72.63689 37.04756, 73.26006 37.49526, 73.94870 37.42157, 74.98000 37.41999, 75.15803 37.13303, 74.57589 37.02084, 74.06755 36.83618, 72.92002 36.72001, 71.84629 36.50994, 71.26235 36.07439, 71.49877 35.65056, 71.61308 35.15320, 71.11502 34.73313, 71.15677 34.34891, 70.88180 33.98886, 69.93054 34.02012, 70.32359 33.35853, 69.68715 33.10550, 69.26252 32.50194, 69.31776 31.90141, 68.92668 31.62019, 68.55693 31.71331, 67.79269 31.58293, 67.68339 31.30315, 66.93889 31.30491, 66.38146 30.73890, 66.34647 29.88794, 65.04686 29.47218, 64.35042 29.56003, 64.14800 29.34082, 63.55026 29.46833, 62.54986 29.31857, 60.87425 29.82924, 61.78122 30.73585, 61.69931 31.37951, 60.94194 31.54807, 60.86365 32.18292, 60.53608 32.98127, 60.96370 33.52883, 60.52843 33.67645, 60.80319 34.40410, 61.21082 35.65007, 62.23065 35.27066, 

In [105]:
df_concepts_geoplot_2 = df_concept_2_processed.groupby(['iso_a3','Concept']).agg(
    **{'Number of mentions': ('iso_a3', 'count')},
    # Concept=('Concept', lambda x: x[x.first_valid_index()] if x.first_valid_index() else None),
    geometry=('geometry', 'first'),
    pop_est=('pop_est', 'first'),
    gdp_md_est=('gdp_md_est', 'first'),
    continent=('continent', 'first'),
    country=('country', 'first'),
).reset_index()

In [106]:
df_concepts_geoplot_2

,iso_a3,Concept,Number of mentions,geometry,pop_est,gdp_md_est,continent,country
0,AFG,FINANCIAL_FLOW,92,"POLYGON ((66.51861 37.36278, 67.07578 37.35614, 67.83000 37.14499, 68.13556 37.02312, 68.85945 37.34434, 69.19627 37.15114, 69.51879 37.60900, 70.11658 37.58822, 70.27057 37.73516, 70.37630 38.13840, 70.80682 38.48628, 71.34813 38.25891, 71.23940 37.95327, 71.54192 37.90577, 71.44869 37.06564, 71.84464 36.73817, 72.19304 36.94829, 72.63689 37.04756, 73.26006 37.49526, 73.94870 37.42157, 74.98000 37.41999, 75.15803 37.13303, 74.57589 37.02084, 74.06755 36.83618, 72.92002 36.72001, 71.84629 36.50994, 71.26235 36.07439, 71.49877 35.65056, 71.61308 35.15320, 71.11502 34.73313, 71.15677 34.34891, 70.88180 33.98886, 69.93054 34.02012, 70.32359 33.35853, 69.68715 33.10550, 69.26252 32.50194, 69.31776 31.90141, 68.92668 31.62019, 68.55693 31.71331, 67.79269 31.58293, 67.68339 31.30315, 66.93889 31.30491, 66.38146 30.73890, 66.34647 29.88794, 65.04686 29.47218, 64.35042 29.56003, 64.14800 29.34082, 63.55026 29.46833, 62.54986 29.31857, 60.87425 29.82924, 61.78122 30.73585, 61.69931 31.37951, 60.94194 31.54807, 60.86365 32.18292, 60.53608 32.98127, 60.96370 33.52883, 60.52843 33.67645, 60.80319 34.40410, 61.21082 35.65007, 62.23065 35.27066, 62.98466 35.40404, 63.19354 35.85717, 63.98290 36.00796, 64.54648 36.31207, 64.74611 37.11182, 65.58895 37.30522, 65.74563 37.66116, 66.21738 37.39379, 66.51861 37.36278))",38041754.0,19291.0,Asia,Afghanistan
1,AGO,FINANCIAL_FLOW,208,"MULTIPOLYGON (((12.99552 -4.78110, 12.63161 -4.99127, 12.46800 -5.24836, 12.43669 -5.68430, 12.18234 -5.78993, 11.91496 -5.03799, 12.31861 -4.60623, 12.62076 -4.43802, 12.99552 -4.78110)), ((12.32243 -6.10009, 12.73517 -5.96568, 13.02487 -5.98439, 13.37560 -5.86424, 16.32653 -5.87747, 16.57318 -6.62264, 16.86019 -7.22230, 17.09000 -7.54569, 17.47297 -8.06855, 18.13422 -7.98768, 18.46418 -7.84701, 19.01675 -7.98825, 19.16661 -7.73818, 19.41750 -7.15543, 20.03772 -7.11636, 20.09162 -6.94309, 20.60182 -6.93932, 20.51475 -7.29961, 21.72811 -7.29087, 21.74646 -7.92008, 21.94913 -8.30590, 21.80180 -8.90871, 21.87518 -9.52371, 22.20875 -9.89480, 22.15527 -11.08480, 22.40280 -10.99308, 22.83735 -11.01762, 23.45679 -10.86786, 23.91222 -10.92683, 24.01789 -11.23730, 23.90415 -11.72228, 24.07991 -12.19130, 23.93092 -12.56585, 24.01614 -12.91105, 21.93389 -12.89844, 21.88784 -16.08031, 22.56248 -16.89845, 23.21505 -17.52312, 21.37718 -17.93064, 18.95619 -17.78909, 18.26331 -17.30995, 14.20971 -17.35310, 14.05850 -17.42338, 13.46236 -16.97121, 12.81408 -16.94134, 12.21546 -17.11167, 11.73420 -17.30189, 11.64010 -16.67314, 11.77854 -15.79382, 12.12358 -14.87832, 12.17562 -14.44914, 12.50010 -13.54770, 12.73848 -13.13791, 13.31291 -12.48363, 13.63372 -12.03864, 13.73873 -11.29786, 13.68638 -10.73108, 13.38733 -10.37358, 13.12099 -9.76690, 12.87537 -9.16693, 12.92906 -8.95909, 13.23643 -8.56263, 12.93304 -7.59654, 12.72830 -6.92712, 12.22735 -6.29445, 12.32243 -6.10009)))",31825295.0,88815.0,Africa,Angola
2,ALB,FINANCIAL_FLOW,368,"POLYGON ((21.02004 40.84273, 20.99999 40.58000, 20.67500 40.43500, 20.61500 40.11001, 20.15002 39.62500, 19.98000 39.69499, 19.96000 39.91501, 19.40608 40.25077, 19.31906 40.72723, 19.40355 41.40957, 19.54003 41.71999, 19.37177 41.87755, 19.37177 41.87755, 19.30449 42.19575, 19.73805 42.68825, 19.80161 42.50009, 20.07070 42.58863, 20.28375 42.32026, 20.52295 42.21787, 20.59025 41.85541, 20.59025 41.85540, 20.46318 41.51509, 20.60518 41.08623, 21.02004 40.84273))",2854191.0,15279.0,Europe,Albania
3,ARE,FINANCIAL_FLOW,44,"POLYGON ((51.57952 24.24550, 51.75744 24.29407, 51.79439 24.01983, 52.57708 24.17744, 53.40401 24.15132, 54.00800 24.12176, 54.69302 24.79789, 55.43902 25.43915, 56.07082 26.05546, 56.26104 25.71461, 56.39685 24.92473, 55.88623 24.92083, 55.80412 24.26960, 55.98121 24.13054, 55.52863 23.93360, 55.52584 23.52487, 55.23449 23.11099, 55.20834 22.70833, 55.00680 22.49695, 52.00073 23.00115, 51.61771 24.01422, 51.57952 24.24550))",9770529.0,42

In [107]:
def add_zero_mentions(df_concepts_geoplot, df_worldbank):
    # for every country in world, check if the concept is in df_concepts_geoplot. If not, add a row with count 0
    for country in df_worldbank["iso_a3"]:
        for conc in df_concepts_geoplot["Concept"].unique():
            if not df_concepts_geoplot[
                (df_concepts_geoplot["iso_a3"] == country) & (df_concepts_geoplot["Concept"] == conc)
            ].empty:
                continue
            else:
                df_concepts_geoplot = pd.concat(
                    [
                        df_concepts_geoplot,
                        df_worldbank[df_worldbank["iso_a3"] == country]
                        .reset_index(drop=True)
                        .merge(
                            pd.DataFrame({"Concept": conc, "Number of mentions": 0}, index=[0]),
                            left_index=True,
                            right_index=True,
                        ),
                    ],
                    ignore_index=True,
                )
    return df_concepts_geoplot

In [108]:
df_concepts_geoplot_1=add_zero_mentions(df_concepts_geoplot_1, df_worldbank)
df_concepts_geoplot_2=add_zero_mentions(df_concepts_geoplot_2, df_worldbank)
# remove Antarctica
df_concepts_geoplot_1 = df_concepts_geoplot_1[df_concepts_geoplot_1["iso_a3"] != "ATA"]
df_concepts_geoplot_2 = df_concepts_geoplot_2[df_concepts_geoplot_2["iso_a3"] != "ATA"]

In [109]:
df_concepts_geoplot_1.head().to_clipboard()

In [110]:
import itertools

# get every possible combination of 2 concepts from the unique concepts in the df_concepts_geoplot_1 and df_concepts_geoplot_2
concepts_1 = df_concepts_geoplot_1["Concept"].unique()
concepts_2 = df_concepts_geoplot_2["Concept"].unique()
concepts = list(itertools.product(concepts_1, concepts_2))
concepts

[('Coal', 'FINANCIAL_FLOW'),
 ('Fossil fuel', 'FINANCIAL_FLOW'),
 ('Gas', 'FINANCIAL_FLOW'),
 ('Oil', 'FINANCIAL_FLOW')]

In [111]:
import pandas as pd

# Merge the two dataframes on 'iso_a3' and 'country'
df = pd.merge(df_concepts_geoplot_1, df_concepts_geoplot_2, on=['iso_a3', 'country'], how='outer')

# Fill NA values with 0 in 'Number of mentions' columns
df[['Number of mentions_x', 'Number of mentions_y']] = df[['Number of mentions_x', 'Number of mentions_y']].fillna(0)

# Create a new column 'combined_concept' which combines 'Concept_x' and 'Concept_y'
df['combined_concept'] = list(zip(df['Concept_x'], df['Concept_y']))

# Create a binary column 'indicator' where True if 'Number of mentions' in both dataframes is > 0, otherwise False
df['indicator'] = (df['Number of mentions_x'] > 0) & (df['Number of mentions_y'] > 0)

# Filter the dataframe based on the list of pairs of concepts
concept_pairs = [('Coal', 'FINANCIAL_FLOW'),
                 ('Fossil fuel', 'FINANCIAL_FLOW'),
                 ('Gas', 'FINANCIAL_FLOW'),
                 ('Oil', 'FINANCIAL_FLOW')]

df_final = df[df['combined_concept'].isin(concept_pairs)][['country', 'combined_concept', 'indicator']]

In [112]:
# Select only the columns with '_x' in their names (i.e., the columns from df1)
df = df[[col for col in df.columns if '_x' in col or col in ['iso_a3', 'country','combined_concept','indicator']]]

# Rename columns to remove '_x'
df.columns = [col.replace('_x', '') for col in df.columns]


In [113]:
df=gpd.GeoDataFrame(df)

In [114]:
def create_choropleth_map_co_occurrence(df, concept):
    df=df[df['combined_concept']==concept]
    concept1, concept2 = concept[0], concept[1]
    concept_str = f"{concept1.title().replace('_',' ')} and {concept2.title().replace('_',' ')}"
    df['binary'] = df['indicator'].apply(lambda x: f"Parties that mention {concept_str}" if x else f"Parties that do not mention {concept_str}")
    choropleth_map = (
        alt.Chart(df[df["combined_concept"] == concept])
        .mark_geoshape(stroke="black", strokeWidth=1)
        .encode(
            color=alt.Color(
                "binary:N",
                scale=alt.Scale(domain=[f"Parties that mention {concept_str}", f"Parties that do not mention {concept_str}"], range=["lightblue", "grey"]),
                legend=alt.Legend(title="Mentioned"),
            ),
            tooltip=["country:N", "pop_est:N", "gdp_md_est:N", "continent:N", "Number of mentions:Q"],
        )
        .properties(
            width=800,
            height=400,
            title=f'UNFCCC Party members mentioning {concept1.title()} and {concept2.title()} in GST submissions',
        )
    )
    return choropleth_map

In [115]:
# conc=concepts[0]
# df_conc = df[df['combined_concept']==conc]
# df_conc['binary'] = df_conc['indicator'].apply(lambda x: f"Parties that mention {conc}" if x else f"Parties that do not mention {conc}")
# # df_conc.iloc[-2]['iso_a3']
# df_conc=df_conc[df_conc.iso_a3!='-99']
# # Filter the dataframe to exclude Antarctica
# df_conc_filtered = df_conc[df_conc["continent"] != "Antarctica"]
#
# # Create the choropleth map with the filtered dataframe
# chart = alt.Chart(df_conc_filtered).mark_geoshape(stroke="black", strokeWidth=1).encode(
#     color=alt.Color(
#         "binary:N",
#         scale=alt.Scale(domain=[f"Parties that mention {conc}", f"Parties that do not mention {conc}"], range=["lightblue", "grey"]),
#         legend=alt.Legend(title="Mentioned"),
#     ),
#     tooltip=["country:N", "pop_est:N", "gdp_md_est:N", "continent:N", "Number of mentions:Q"],
# ).properties(
#     width=800,
#     height=400,
#     title=f'UNFCCC Party members mentioning {concepts[0]} in GST submissions',
# )
#
# chart

In [116]:
# Iterate over the unique concepts and create a separate choropleth map for each
for conc in concepts:
    choropleth_map = create_choropleth_map_co_occurrence(df, conc)
    choropleth_map.display()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [117]:
df

,iso_a3,Concept,Number of mentions,geometry,pop_est,gdp_md_est,continent,country,name,combined_concept,indicator
0,AFG,Coal,68.0,"POLYGON ((66.51861 37.36278, 67.07578 37.35614, 67.83000 37.14499, 68.13556 37.02312, 68.85945 37.34434, 69.19627 37.15114, 69.51879 37.60900, 70.11658 37.58822, 70.27057 37.73516, 70.37630 38.13840, 70.80682 38.48628, 71.34813 38.25891, 71.23940 37.95327, 71.54192 37.90577, 71.44869 37.06564, 71.84464 36.73817, 72.19304 36.94829, 72.63689 37.04756, 73.26006 37.49526, 73.94870 37.42157, 74.98000 37.41999, 75.15803 37.13303, 74.57589 37.02084, 74.06755 36.83618, 72.92002 36.72001, 71.84629 36.50994, 71.26235 36.07439, 71.49877 35.65056, 71.61308 35.15320, 71.11502 34.73313, 71.15677 34.34891, 70.88180 33.98886, 69.93054 34.02012, 70.32359 33.35853, 69.68715 33.10550, 69.26252 32.50194, 69.31776 31.90141, 68.92668 31.62019, 68.55693 31.71331, 67.79269 31.58293, 67.68339 31.30315, 66.93889 31.30491, 66.38146 30.73890, 66.34647 29.88794, 65.04686 29.47218, 64.35042 29.56003, 64.14800 29.34082, 63.55026 29.46833, 62.54986 29.31857, 60.87425 29.82924, 61.78122 30.73585, 61.69931 31.37951, 60.94194 31.54807, 60.86365 32.18292, 60.53608 32.98127, 60.96370 33.52883, 60.52843 33.67645, 60.80319 34.40410, 61.21082 35.65007, 62.23065 35.27066, 62.98466 35.40404, 63.19354 35.85717, 63.98290 36.00796, 64.54648 36.31207, 64.74611 37.11182, 65.58895 37.30522, 65.74563 37.66116, 66.21738 37.39379, 66.51861 37.36278))",38041754.0,19291.0,Asia,Afghanistan,NaN,"(Coal, FINANCIAL_FLOW)",True
1,AFG,Fossil fuel,2.0,"POLYGON ((66.51861 37.36278, 67.07578 37.35614, 67.83000 37.14499, 68.13556 37.02312, 68.85945 37.34434, 69.19627 37.15114, 69.51879 37.60900, 70.11658 37.58822, 70.27057 37.73516, 70.37630 38.13840, 70.80682 38.48628, 71.34813 38.25891, 71.23940 37.95327, 71.54192 37.90577, 71.44869 37.06564, 71.84464 36.73817, 72.19304 36.94829, 72.63689 37.04756, 73.26006 37.49526, 73.94870 37.42157, 74.98000 37.41999, 75.15803 37.13303, 74.57589 37.02084, 74.06755 36.83618, 72.92002 36.72001, 71.84629 36.50994, 71.26235 36.07439, 71.49877 35.65056, 71.61308 35.15320, 71.11502 34.73313, 71.15677 34.34891, 70.88180 33.98886, 69.93054 34.02012, 70.32359 33.35853, 69.68715 33.10550, 69.26252 32.50194, 69.31776 31.90141, 68.92668 31.62019, 68.55693 31.71331, 67.79269 31.58293, 67.68339 31.30315, 66.93889 31.30491, 66.38146 30.73890, 66.34647 29.88794, 65.04686 29.47218, 64.35042 29.56003, 64.14800 29.34082, 63.55026 29.46833, 62.54986 29.31857, 60.87425 29.82924, 61.78122 30.73585, 61.69931 31.37951, 60.94194 31.54807, 60.86365 32.18292, 60.53608 32.98127, 60.96370 33.52883, 60.52843 33.67645, 60.80319 34.40410, 61.21082 35.65007, 62.23065 35.27066, 62.98466 35.40404, 63.19354 35.85717, 63.98290 36.00796, 64.54648 36.31207, 64.74611 37.11182, 65.58895 37.30522, 65.74563 37.66116, 66.21738 37.39379, 66.51861 37.36278))",38041754.0,19291.0,Asia,Afghanistan,NaN,"(Fossil fuel, FINANCIAL_FLOW)",True
2,AFG,Gas,125.0,"POLYGON ((66.51861 37.36278, 67.07578 37.35614, 67.83000 37.14499, 68.13556 37.02312, 68.85945 37.34434, 69.19627 37.15114, 69.51879 37.60900, 70.11658 37.58822, 70.27057 37.73516, 70.37630 38.13840, 70.80682 38.48628, 71.34813 38.25891, 71.23940 37.95327, 71.54192 37.90577, 71.44869 37.06564, 71.84464 36.73817, 72.19304 36.94829, 72.63689 37.04756, 73.26006 37.49526, 73.94870 37.42157, 74.98000 37.41999, 75.15803 37.13303, 74.57589 37.02084, 74.06755 36.83618, 72.92002 36.72001, 71.84629 36.50994, 71.26235 36.07439, 71.49877 35.65056, 71.61308 35.15320, 71.11502 34.73313, 71.15677 34.34891, 70.88180 33.98886, 69.93054 34.02012, 70.32359 33.35853, 69.68715 33.10550, 69.26252 32.50194, 69.31776 31.90141, 68.92668 31.62019, 68.55693 31.71331, 67.79269 31.58293, 67.68339 31.30315, 66.93889 31.30491, 66.38146 30.73890, 66.34647 29.88794, 65.04686 29.47218, 64.35042 29.56003, 64.14800 29.34082, 63.55026 29.46833, 62.54986 29.31857, 60.87425 29.82924, 61.78122 30.73585, 61.69931 31.37951, 60.94194 31.54807, 60.86365 32.18292, 6

In [118]:
df_concept_1_processed

text  \
0      processing activities. The extraction, production, processing, or transportation of fuels involve substantial quantity of methane emissions to the atmosphere. Oil and natural gas systems are potentially very complex and diverse. Two major issues concerning the reported fugitive emissions from oil and gas systems are: (i) the generally poor quality and completeness of available venting and flaring data; (ii) the fact that much of the infrastructure contributing to equipment leaks is at minor facilities for which statistics are either unavailable or incomplete (e.g. well site facilities and field facilities). Fugitive emissions consist of CH, mostly however CO, is also emitted from the coal mines. It is the first time that the government actually started to use coal for electricity generation. It was assumed that in 2017-18 most of the coal mines were underground (80%) and 20% coal was extracted from the surface mines. There are oil and gas production, processing, transmission and distribution activities so around 3.29 Mt CO2e emission from oil and gas related infrastructure. Figure 2.4 shows the fugitive GHG emissions.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    In the category of Fugitive Emissions from Fuels, CO₂ and CH₂ emissions from the manufacture of fuels, generated by obtaining primary fuels and the manufacture of secondary fuels, were considered. It is considered both for extraction and handling of mineral coal, as well as emissions from oil and natural gas activities. Fugitive emissions from Oil and Natural Gas activities [182] represent 97.86% (3,288 Gg CO₂eq), while those of Solid Fuels [181] 214% (72 Gg CO eq). Fugitive emissions are derived from venting processes, flaring, evaporative losses, accidental releases, as well as processing, transmission, and storage.   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   According to the IPCC methodology, emissions coming from energy activities like fugitive emissions from fuels are calculated considering the intentional and accidental releases of gases from anthropogenic activities. In this group are included: Solid Fuels (activities concerning coal underground mining and post mining activities, as well as solid fuel transformation activities); Oil & Natural Gas (exploitation, production/processing, transmission/distribution, refining, other leakage) and Oil & Natural Gas Venting & Flaring.   
3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [119]:
# We want to plot the number of authors mentioning both.
total_stakeholders = df_concepts_combined["author"].nunique()
num_stakeholders_mentioning = df_concepts_combined.groupby(["Concept", "category"])[
    "author"
].nunique()
percentage_stakeholders_mentioning = (
    num_stakeholders_mentioning / total_stakeholders * 100
)

In [120]:
total_stakeholders

304

In [121]:
num_stakeholders_mentioning

Concept         category 
Coal            Non-Party     49
                Party        162
FINANCIAL_FLOW  Non-Party    104
                Party        199
Fossil fuel     Non-Party      6
                Party        100
Gas             Non-Party     47
                Party        193
Oil             Non-Party     41
                Party        192
Name: author, dtype: int64

In [122]:
df_concepts_combined.head().drop(columns="geometry").head().to_clipboard()
df=df_concepts_combined.copy()

In [123]:
df[df.category=='Non-Party']

text  \
16                                                                                                                                                                                                                                                                                                                          Potential improvements within source category 1B2 'Fugitive Emissions from Oil and Natural Gas' could be possible should new data related to fugitive leaks from oil and natural gas distribution networks be available (from the infrastructure needed to produce, collect, process, refine and distribute oil products and natural gases to final consumers; from equipment functioning, evaporation and flashing losses, flaring, accidental releases from pipeline dig-ins, etc.); respectively, in the case of adopting a higher-ranking assessment methodology. The possibility to obtain AD associated to LPG consumption on the LBDR for the entire period under review will also be estimated.   
17                                                                                                                                                                                                                                                                                                                                                                                    • Availability of additional data on fugitive leaks from oil and natural gas distribution networks (from the infrastructure needed to produce, collect, process, refine and distribute oil products and natural gases to final consumers; from equipment functioning, evaporation and flashing losses, flaring, accidental releases from pipeline dig-ins, etc.) (source category 1B2 'Fugitive Emissions from Oil and Natural Gas), i.e., in the case of adopting a higher-ranking assessment methodology, the possibility to obtain AD associated to LPG consumption on the LBDR for the entire period under review will also be estimated;   
19                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Petróleos Mexicanos participates as a company in important climate initiatives and, relevantly, in the Global Methane Initiative, in the Association for the Reduction of Global Gas Flaring Reduction Partnership, and has recently joined joined the Oil and Gas Climate Initiative; As part of the latter, along with ten other major oil companies in the world, PEMEX has voluntarily committed to:   
30                                                                                                                                                                                                                                                                                                                                                                  Most of the national market is supplied by the Société Gabonaise de Raffinerie (SOGARA), but according to the data collected from the SGEPP, there is a flow of imports/exports of petroleum products. Crude production reached 449,708.92 TJ in 2017, that of natural gas 102,086 TJ and for electricity generation needs, 18,824.43 T] of fossil fuels were consumed. But 80% of the crude is exported while for NG part of the treated gas is consumed by the local market for the generation of electricity and another part of the mixed gas is either flared or consumed as lift gas for the extraction of crude oil and as fuel for the energy industry.

In [124]:
import pandas as pd
from itertools import combinations

# Assuming your DataFrame is named "df"
# and the three columns of interest are "author", "Concept", and "category"

# Group by category, concept, and author to count unique authors
grouped = df.groupby(['category', 'Concept', 'author']).size().reset_index().groupby(['category', 'Concept']).size()

In [125]:
df.groupby(['category', 'Concept', 'author']).size().reset_index().groupby(['category', 'Concept']).size()

category   Concept       
Non-Party  Coal               49
           FINANCIAL_FLOW    104
           Fossil fuel         6
           Gas                47
           Oil                41
Party      Coal              162
           FINANCIAL_FLOW    199
           Fossil fuel       100
           Gas               193
           Oil               192
dtype: int64

In [126]:
grouped

category   Concept       
Non-Party  Coal               49
           FINANCIAL_FLOW    104
           Fossil fuel         6
           Gas                47
           Oil                41
Party      Coal              162
           FINANCIAL_FLOW    199
           Fossil fuel       100
           Gas               193
           Oil               192
dtype: int64

In [127]:
import pandas as pd
from itertools import combinations

# Assuming your DataFrame is named "df"
# and the three columns of interest are "author", "Concept", and "category"

# Group by category, concept, and author to count unique authors
grouped = df.groupby(['category', 'author'])['Concept'].apply(set).reset_index()

In [128]:
grouped

,category,author,Concept
0,Non-Party,Adaptation Research Alliance (ARA),{FINANCIAL_FLOW}
1,Non-Party,African Group Submission for the Second Technical Dialogue of the Global Stocktake,"{FINANCIAL_FLOW, Gas}"
2,Non-Party,Alliance of Small Island Developing States,{FINANCIAL_FLOW}
3,Non-Party,CAD2,{FINANCIAL_FLOW}
4,Non-Party,CDP Worldwide,{FINANCIAL_FLOW}
...,...,...,...
303,Party,Venezuela (Bolivarian Republic of),"{Oil, Gas, Coal, Fossil fuel, FINANCIAL_FLOW}"
304,Party,Viet Nam,"{FINANCIAL_FLOW, Oil, Gas, Coal}"
305,Party,Yemen,"{FINANCIAL_FLOW, Oil, Gas, Coal}"
306,Party,Zambia,"{FINANCIAL_FLOW, Oil, Gas, Coal}"


In [ ]:

# Get all unique categories and concepts
categories = df['category'].unique()
concepts = df['Concept'].unique()

In [146]:
concept_groups=df.groupby('concept_group')['Concept'].unique()
concepts_group_1=concept_groups.iloc[0].tolist()
concepts_group_2=concept_groups.iloc[1].tolist()

In [148]:
concepts_group_1

['Gas', 'Coal', 'Oil', 'Fossil fuel']

In [149]:
# all combinations of 2 concepts with one from each group
concept_pairs = [(a, b) for a in concepts_group_1 for b in concepts_group_2]


[('Gas', 'FINANCIAL_FLOW'),
 ('Coal', 'FINANCIAL_FLOW'),
 ('Oil', 'FINANCIAL_FLOW'),
 ('Fossil fuel', 'FINANCIAL_FLOW')]

In [150]:


# Create a new DataFrame to store the results
result_df = pd.DataFrame(columns=['category', 'Concepts', 'UniqueAuthors'])

# Iterate over all combinations of concepts
for category in categories:
    category_authors = grouped[grouped['category'] == category]['author']

    # Generate all combinations of concepts from group 1 and group 2
    concept_pairs = [(a, b) for a in concepts_group_1 for b in concepts_group_2]


    for concepts in concept_pairs:
        concept1, concept2 = concepts

        # Count unique authors that mention both concepts
        unique_authors = 0
        for author in category_authors:
            author_concepts = grouped[(grouped['category'] == category) & (grouped['author'] == author)]['Concept'].values[0]
            if concept1 in author_concepts and concept2 in author_concepts:
                unique_authors += 1

        result_df = result_df.append({'category': category, 'Concepts': concepts, 'UniqueAuthors': unique_authors}, ignore_index=True)

print(result_df)


    category                       Concepts UniqueAuthors
0      Party          (Gas, FINANCIAL_FLOW)           193
1      Party         (Coal, FINANCIAL_FLOW)           162
2      Party          (Oil, FINANCIAL_FLOW)           192
3      Party  (Fossil fuel, FINANCIAL_FLOW)           100
4  Non-Party          (Gas, FINANCIAL_FLOW)            42
5  Non-Party         (Coal, FINANCIAL_FLOW)            47
6  Non-Party          (Oil, FINANCIAL_FLOW)            41
7  Non-Party  (Fossil fuel, FINANCIAL_FLOW)             6


In [151]:
result_df_percentage = result_df.copy()
result_df_percentage['UniqueAuthors'] = result_df_percentage['UniqueAuthors'] / total_stakeholders * 100

In [152]:
df_counts_and_percentages = pd.merge(result_df, result_df_percentage, on=['category', 'Concepts'])
df_counts_and_percentages = df_counts_and_percentages.rename(columns={'UniqueAuthors_x':'count', 'UniqueAuthors_y':'percentage'})
df_counts_and_percentages['Concept'] = df_counts_and_percentages['Concepts'].apply(lambda x: f"{x[0].title().replace('_',' ')} and {x[1].title().replace('_',' ')}")
df_counts_and_percentages

,category,Concepts,count,percentage,Concept
0,Party,"(Gas, FINANCIAL_FLOW)",193,63.486842,Gas and Financial Flow
1,Party,"(Coal, FINANCIAL_FLOW)",162,53.289474,Coal and Financial Flow
2,Party,"(Oil, FINANCIAL_FLOW)",192,63.157895,Oil and Financial Flow
3,Party,"(Fossil fuel, FINANCIAL_FLOW)",100,32.894737,Fossil Fuel and Financial Flow
4,Non-Party,"(Gas, FINANCIAL_FLOW)",42,13.815789,Gas and Financial Flow
5,Non-Party,"(Coal, FINANCIAL_FLOW)",47,15.460526,Coal and Financial Flow
6,Non-Party,"(Oil, FINANCIAL_FLOW)",41,13.486842,Oil and Financial Flow
7,Non-Party,"(Fossil fuel, FINANCIAL_FLOW)",6,1.973684,Fossil Fuel and Financial Flow


In [155]:
def create_stacked_chart(df, title):
    # Create a bar chart for the count
    bar_count = alt.Chart(df).mark_bar().encode(
        alt.X('Concept:N', title=None),
        alt.Y('count:Q', axis=alt.Axis(title='Count')),
        color=alt.Color('category:N', scale=alt.Scale(domain=['Party', 'Non-Party'], range=['lightblue', 'darkblue']), legend=alt.Legend(title="Category"))
    )

    # Create a text chart for the percentage
    text_percentage = alt.Chart(df).mark_text(dy=-5).encode(
        alt.X('Concept:N'),
        alt.Y('percentage:Q'),
        color=alt.value('black')
    )

    # Combine the charts
    chart = alt.layer(bar_count, text_percentage).resolve_scale(y='independent')

    chart = chart.properties(title=title)
    return chart

create_stacked_chart(df_counts_and_percentages, f"Number and percentage of UNFCCC Party and Non-Party documents that mention related concepts in submitted documents")

alt.LayerChart(...)

In [158]:
display(Markdown(f"## Number and percentage of UNFCCC Party and Non-Party documents that mention {concept_1} in submitted documents"))

## Number and percentage of UNFCCC Party and Non-Party documents that mention fossil-fuels in submitted documents

In [191]:
def prepare_data(df, value_col, total=None):
    if total_stakeholders is not None:
        df = df / total_stakeholders * 100
    df = df.reset_index().rename(columns={value_col: "count"})
    df["count"] = df["count"].astype(int)
    df["category"] = df["category"].str.replace("Non-Party Stakeholder", "Non-Party")
    df["category"] = df["category"].str.replace("Party Stakeholder", "Party")
    return df

def create_count_and_percentage_df(df):
    total_stakeholders = df["author"].nunique()
    num_stakeholders_mentioning = df.groupby(["Concept", "category"])[
        "author"
    ].nunique()
    percentage_stakeholders_mentioning = (
        num_stakeholders_mentioning / total_stakeholders * 100
    )

    num_stakeholders_data = prepare_data(num_stakeholders_mentioning, "author")
    percentage_stakeholders_data = prepare_data(
        percentage_stakeholders_mentioning, "author", total=total_stakeholders
    )
    percentage_stakeholders_data = percentage_stakeholders_data.rename(
        columns={"count": "percentage"}
    )

    df_counts_and_percentages = num_stakeholders_data.merge(percentage_stakeholders_data, on=["Concept", "category"])
    return df_counts_and_percentages

df_counts_and_percentages_1 = create_count_and_percentage_df(df_concept_1_processed)
df_counts_and_percentages_2 = create_count_and_percentage_df(df_concept_2_processed)

In [192]:
create_stacked_chart(df_counts_and_percentages_1,'hi')

alt.LayerChart(...)

In [159]:
display(Markdown(f"## Number and percentage of UNFCCC Party and Non-Party documents that mention {concept_2} in submitted documents"))

## Number and percentage of UNFCCC Party and Non-Party documents that mention financial-flows in submitted documents

In [194]:
display(Markdown(f"## Document types that mention {concept_1} and {concept_2}"))
create_stacked_chart(df_counts_and_percentages_2,'hi')

## Document types that mention fossil-fuels and financial-flows

alt.LayerChart(...)

In [164]:
df_concepts_combined

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                processing activities. The extraction, production, processing, or transportation of fuels involve substantial quantity of methane emissions to the atmosphere. Oil and natural gas systems are potentially very complex and diverse. Two major issues concerning the reported fugitive emissions from oil and gas systems are: (i) the generally poor quality and completeness of available venting and flaring data; (ii) the fact that much of the infrastructure contributing to equipment leaks is at minor facilities for which statistics are either unavailable or incomplete (e.g. well site facilities and field facilities). Fugitive emissions consist of CH, mostly however CO, is also emitted from the coal mines. It is the first time that the government actually started to use coal for electricity generation. It was assumed that in 2017-18 most of the coal mines were underground (80%) and 20% coal was extracted from the surface mines. There are oil and gas production, processing, transmission and distribution activities so around 3.29 Mt CO2e emission from oil and gas related infrastructure. Figure 2.4 shows the fugitive GHG emissions.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [165]:
concept_pairs

[('Gas', 'FINANCIAL_FLOW'),
 ('Coal', 'FINANCIAL_FLOW'),
 ('Oil', 'FINANCIAL_FLOW'),
 ('Fossil fuel', 'FINANCIAL_FLOW')]

In [176]:
dd={}
for category_pair in concept_pairs:
    # count unique documents that mention both concepts
    unique_documents = 0
    for document in df_concepts_combined['document_id'].unique():
        document_concepts = df_concepts_combined[df_concepts_combined['document_id'] == document]['Concept'].values
        if category_pair[0] in document_concepts and category_pair[1] in document_concepts:
            unique_documents += 1
    dd[category_pair]=unique_documents




In [177]:
dd

{('Gas', 'FINANCIAL_FLOW'): 604,
 ('Coal', 'FINANCIAL_FLOW'): 503,
 ('Oil', 'FINANCIAL_FLOW'): 592,
 ('Fossil fuel', 'FINANCIAL_FLOW'): 208}

In [185]:
df = pd.DataFrame.from_dict(dd, orient='index', columns=['count']).reset_index()

In [187]:
df.columns=['Concept','count']

In [190]:

# create the Altair chart
chart = alt.Chart(df).mark_bar(color='darkblue').encode(
    x='Concept:N',
    y='count:Q',
).properties(
    title='Count of Documents by Concept and Category'
)

chart

alt.Chart(...)

In [ ]:
# plot the number of documents that mention both concepts


In [179]:
df_concepts_combined.document_id.nunique()

807